In [64]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import warnings
import scipy.stats as stats
# Ignore warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder as ore

In [65]:
df = pd.read_csv("cleaned_df.csv")

In [110]:

X = df.drop(columns=["G3"])
y = df['G3']


# Model Building

## First we will define our features and our target

In [67]:
def denormalize(y):#this is a denormalize function
    u=np.mean(df['G3'])
    sigma=np.std(df['G3'])
    for i in y:
        i=i*sigma+u
    return y

In [150]:
def significance_hypothesis_test(X,y_train,y_pred,coeff):
    N=len(y_train)
    numerator=np.sqrt(mean_squared_error(y_train,y_pred)*(N/(N-2)))
    SE=[]
   
    cols=X.columns.copy()
    for column in cols:
        denomenator=np.sqrt(np.std(X[column])*N)
        SE.append(numerator/denomenator)
    Z_score=[]
    p_score=[]
    new_coeff=[]
  

    

    for i in range(len(coeff)):
        Z_score.append(coeff[i]/SE[i])
        p_score.append(2*stats.norm.cdf(-abs(Z_score[i])))
        if(p_score[i]<0.05):
            new_coeff.append((coeff[i],X.columns[i]))
        else:
            new_coeff.append((0,X.columns[i]))
    return new_coeff,p_score,Z_score
    
    

In [69]:
df = pd.read_csv("cleaned_df.csv")

In [70]:
pipe=Pipeline([('ore',ore()),('Normalizer',Normalizer()),('Ridge',Ridge())])


In [151]:
pipe.fit(X,y)
y_pred_norm=pipe.predict(X)
y_pred=denormalize(y_pred_norm)
mse = mean_squared_error(y, y_pred)
print("MSE Score: ",mse)
rs=r2_score(y,y_pred)
print("R^2 Score: ",rs)
enc=ore()

X1=enc.fit_transform(X)

encoded_df = pd.DataFrame(X1,columns=enc.get_feature_names_out(X.columns))

nc,ps,zs=significance_hypothesis_test(encoded_df,y,y_pred,pipe.named_steps['Ridge'].coef_)
print(sorted(nc))
print(ps)
print(zs)


MSE Score:  1.9372166794863535
R^2 Score:  0.7158669150317206
[(-4.251631314929467, 'Fjob'), (-4.24955134687962, 'Pstatus'), (-4.246244464433095, 'health'), (-2.8268549964940477, 'school'), (-2.796755298942414, 'freetime'), (-2.350315197494154, 'higher'), (-2.317099544752715, 'famsup'), (-2.23694409223507, 'goout'), (-2.133913356917573, 'guardian'), (-1.9783341842013586, 'famrel'), (-1.966770282517515, 'schoolsup'), (-1.9205200199160264, 'romantic'), (-1.9031221880883968, 'famsize'), (-1.8065716042626958, 'nursery'), (-1.7465399653503237, 'address'), (-1.586755436045184, 'traveltime'), (-1.5483695454272688, 'Fedu'), (-1.2303925922255434, 'sex'), (-1.2237898803374019, 'failures'), (-1.2094972492308063, 'Walc'), (-1.2010520210302487, 'activities'), (-1.0311726293682784, 'absences'), (-0.9807171373741402, 'internet'), (-0.9374703724503006, 'reason'), (-0.8056896158240379, 'Dalc'), (-0.634834404141638, 'paid'), (-0.39803660486462256, 'studytime'), (0, 'Medu'), (0, 'age'), (0.55410871500276

Here we tried Ridge regression with cross 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
import numpy as np
alphas = [0,0.1, 1.0, 10.0]
mean_cv_scores = []
for alpha in alphas:
    # Define the pipeline with Ridge Regression
    pipeline = Pipeline([
        ('ore',ore()),
        ('scaler', StandardScaler()),
        ('ridge', Ridge(alpha=alpha))
    ])
        # Define K-Fold Cross-Validation
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    # Perform K-Fold Cross-Validation
    cv_scores = cross_val_score(pipeline, X, y, cv=kfold)
    mean_cv_score = np.mean(cv_scores)
    mean_cv_scores.append(mean_cv_score)
best_alpha = alphas[np.argmax(mean_cv_scores)]
pipe=Pipeline([('ore',ore()),('Normalizer',Normalizer()),('Ridge',Ridge(alpha=best_alpha))])
pipe.fit(X,y)
y_pred_norm=pipe.predict(X)
y_pred=denormalize(y_pred_norm)
mse = mean_squared_error(y, y_pred)
print("MSE Score: ",mse)
rs=r2_score(y,y_pred)
print("R^2 Score: ",rs)


MSE Score:  1.676226900577083
R^2 Score:  0.7541464899558548


Here we tried only linear regression with the feature we made from previous grades

In [152]:
X=df[['avg_Grade']]
y=df['G3']
x_train, x_test, y_train, y_test = train_test_split(X, y ,random_state = 104, train_size=0.8, shuffle=True) 

model=LinearRegression()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
# Calculate mean and standard deviation of MSE and R2
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean MSE: {:.2f}".format(mse))
print("Mean R^2: {:.2f}".format(r2))


y_pred_train = model.predict(x_train)
train_r2 = r2_score(y_train, y_pred_train)
print("Train R^2 Score:", train_r2)

# Evaluate the best model on the test set
y_pred_test = model.predict(x_test)
test_r2 = r2_score(y_test, y_pred_test)
print("Test R^2 Score:", test_r2)
nc,ps,zs=significance_hypothesis_test(X,y_t,y_pred,model.coef_)
print(nc)
print(ps)
print(zs)


Mean MSE: 0.93
Mean R^2: 0.88
Train R^2 Score: 0.8775947730876035
Test R^2 Score: 0.8791414363302177
[(0.9438224181412738, 'avg_Grade')]
[4.440419873752014e-50]
[14.880068898777415]


Trying Linear Regression with encoding categorical variables

In [ ]:
l = []
cat = []

for i in df:
    if (type(df[i][0]) != str):
        l.append(i)
    else:
        cat.append(i)
        
l.remove("G3")
x = df[l].copy()
cat = df[cat].copy() # cat is df

for i in cat:
    unique = cat[i].unique()
    uniqueDict = dict()
      
    for c in range(len(unique)):
        uniqueDict[unique[c]] = c
        
    cat[i] = cat[i].apply(lambda j: uniqueDict[j])
        
    # for j in cat[i].keys():
    #     cat.loc[j, i] = uniqueDict[cat[i][j]]
        
    cat[i] = cat[i].astype("int64")
    x[i] = cat[i].copy()

for i in x:
    u = np.mean(x[i])
    sigma = np.std(x[i])
    x[i] = (x[i] - u) / sigma

y = df["G3"].copy()

for i in y.keys():
    u = np.mean(y)
    sigma = np.std(y)
    y[i] = (y[i] - u) / sigma

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y ,random_state = 104, train_size=0.8, shuffle=True) 

regr = LinearRegression(fit_intercept = True)
regr.fit(x_train, y_train)
y_pred = regr.predict(x_test)

print("MSE Score: ",mean_squared_error(y_test, y_pred))

y_pred_train = regr.predict(x_train)
train_r2 = r2_score(y_train, y_pred_train)
print("Train R^2 Score:", train_r2)

# Evaluate the best model on the test set
y_pred_test = regr.predict(x_test)
test_r2 = r2_score(y_test, y_pred_test)
print("Test R^2 Score:", test_r2)


MSE Score:  0.4791093138804456
Train R^2 Score: 0.7088781113732967
Test R^2 Score: 0.7548654187190122


Here we selected the columns with the highest correlation with the target and the rejected column from our hypothesis testing

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder  as ore
from sklearn.preprocessing import OneHotEncoder  as ohe
df_copy = df.copy()
binary_encoder = LabelEncoder()
# These are the columns affecting the target from our hypothesis testing
columns = ['school', 'sex', 'address', 'schoolsup', 'higher', 'internet']
for col in columns:
    df_copy[f'{col}2'] = binary_encoder.fit_transform(df[f'{col}'])
df_copy.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,health,absences,G3,avg_Grade,school2,sex2,address2,schoolsup2,higher2,internet2
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,3,4,11,5.5,0,0,1,1,1,0
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,3,2,11,10.0,0,0,1,0,1,1
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,3,6,12,12.5,0,0,1,1,1,1
3,GP,F,15,U,GT3,T,4,2,health,services,...,5,0,14,14.0,0,0,1,0,1,1
4,GP,F,16,U,GT3,T,3,3,other,other,...,5,0,13,12.0,0,0,1,0,1,0


In [ ]:
columns2=[f"{c}2"for c in columns]
# These features have strong correlation with our targeet variable
columns2.extend(['avg_Grade',"Dalc","failures","Medu","studytime"])
columns2

['school2',
 'sex2',
 'address2',
 'schoolsup2',
 'higher2',
 'internet2',
 'avg_Grade',
 'Dalc',
 'failures',
 'Medu',
 'studytime']

Applying Grid search to test different parameters for our ridge model

In [154]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

X = df_copy[columns2]
y = df_copy['G3']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Ridge()

param_grid = {
    'alpha': [0.1, 1.0, 10.0],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best MSE Score:", -grid_search.best_score_)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", test_mse)

y_pred_train = best_model.predict(X_train)
train_r2 = r2_score(y_train, y_pred_train)
print("Train R^2 Score:", train_r2)

# Evaluate the best model on the test set
y_pred_test = best_model.predict(X_test)
test_r2 = r2_score(y_test, y_pred_test)
print("Test R^2 Score:", test_r2)
nc,ps,zs=significance_hypothesis_test(X,y_test,y_pred,best_model.coef_)
print(nc)
print(ps)
print(zs)


Best Parameters: {'alpha': 10.0, 'solver': 'sparse_cg'}
Best MSE Score: 0.7826530227354557
Test MSE: 0.9693272722517692
Train R^2 Score: 0.8869368045277655
Test R^2 Score: 0.8711625341513162
[(0, 'school2'), (0, 'sex2'), (0, 'address2'), (-0.40016172888413376, 'schoolsup2'), (0, 'higher2'), (0, 'internet2'), (0.9385614704549294, 'avg_Grade'), (0, 'Dalc'), (0, 'failures'), (0, 'Medu'), (0, 'studytime')]
[0.8497351360693214, 0.7946260447686915, 0.9156655903134465, 0.03412901032035006, 0.7133254273309061, 0.6797771428355013, 1.4471951137938119e-47, 0.9571141341575016, 0.4844525583689884, 0.4261551544406482, 0.30523620206470725]
[0.18945638454847444, -0.26030820405498045, 0.10589508932379808, -2.118544145824051, 0.3673936234089718, 0.4127672572551604, 14.487792484444284, -0.05377536856081464, -0.6991591788474649, 0.7957881948552687, 1.025269162561113]


Trying KNeighborsRegressor with Grid search to find the best parameters

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error,r2_score

# Assuming df_copy contains your DataFrame and columns2 are the features
X = df_copy[columns2]
y = df_copy['G3']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create KNN Regressor model
model = KNeighborsRegressor()

# Define hyperparameters to search
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
}

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Print the best parameters and best MSE score
print("Best Parameters:", grid_search.best_params_)
print("Best MSE Score:", -grid_search.best_score_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", test_mse)


Best Parameters: {'n_neighbors': 11, 'p': 2, 'weights': 'uniform'}
Best MSE Score: 0.9217446206537115
Test MSE: 1.0117302052785921


In [ ]:
y_pred_train = best_model.predict(X_train)
train_r2 = r2_score(y_train, y_pred_train)
print("Train R^2 Score:", train_r2)

# Evaluate the best model on the test set
y_pred_test = best_model.predict(X_test)
test_r2 = r2_score(y_test, y_pred_test)
print("Test R^2 Score:", test_r2)


Train R^2 Score: 0.8845132058921631
Test R^2 Score: 0.8655265775532557


In [ ]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
numerical_columns = df.select_dtypes(include=['int64','Float64'])


In [ ]:
X = numerical_columns.drop(columns=["G3"])
y = numerical_columns["G3"]

In [ ]:
X = df_copy[columns2]
y = df_copy['G3']

# Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Trying Ensemblle Learning boosting techniqe called Xgboost with hyperparameter tuning and Kfold cross validation

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import mean_squared_error


# Define the XGBoost regressor
model = xgb.XGBRegressor()

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [0, 0.1, 0.5, 1]
}

# Perform cross-validation with hyperparameter tuning
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=20, scoring='neg_mean_squared_error', cv=kfold, verbose=1, random_state=42)

# Fit the model
random_search.fit(X, y)

# Print the best hyperparameters found
print("Best hyperparameters:", random_search.best_params_)

# Print the mean squared error of the best estimator
print("Best MSE:", -random_search.best_score_)

# You can also access the best estimator directly
best_model = random_search.best_estimator_


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best hyperparameters: {'subsample': 0.8, 'reg_lambda': 0.1, 'reg_alpha': 0.1, 'n_estimators': 300, 'max_depth': 3, 'learning_rate': 0.01, 'colsample_bytree': 0.9}
Best MSE: 0.850710649211979


Using the parameters from our best model

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

# Initialize XGBoost Regressor
model = xgb.XGBRegressor(
   n_estimators=100,  
   max_depth=5,       
   learning_rate=0.1, 
   subsample=0.8,     
   colsample_bytree=0.8, 
   random_state=42    
)

# Use a list to store scores
mse_scores = []
r2_scores = []

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Calculate mean and standard deviation of MSE and R2
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean MSE: {:.2f}".format(mse))
print("Mean R^2: {:.2f}".format(r2))


y_pred_train = model.predict(X_train)
train_r2 = r2_score(y_train, y_pred_train)
print("Train R^2 Score:", train_r2)

# Evaluate the best model on the test set
y_pred_test = model.predict(X_test)
test_r2 = r2_score(y_test, y_pred_test)
print("Test R^2 Score:", test_r2)


Mean MSE: 1.13
Mean R^2: 0.85
Train R^2 Score: 0.9603187838841297
Test R^2 Score: 0.8503711139456096


The final model appears to be overfitting the training data

In [165]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

# Initialize XGBoost Regressor
model = xgb.XGBRegressor(
   n_estimators=100,  
   max_depth=3,       
   learning_rate=0.1, 
   subsample=0.9,    
   reg_alpha=5, 
   reg_lambda=5,
   colsample_bytree=0.7, 
   random_state=42    
)

# Use a list to store scores
mse_scores = []
r2_scores = []

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Calculate mean and standard deviation of MSE and R2
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean MSE: {:.2f}".format(mse))
print("Mean R^2: {:.2f}".format(r2))


y_pred_train = model.predict(X_train)
train_r2 = r2_score(y_train, y_pred_train)
print("Train R^2 Score:", train_r2)

# Evaluate the best model on the test set
y_pred_test = model.predict(X_test)
test_r2 = r2_score(y_test, y_pred_test)
print("Test R^2 Score:", test_r2)
nc,ps,zs=significance_hypothesis_test(X,y_test,y_pred,best_model.coef_)
print(nc)
print(ps)
print(zs)


Mean MSE: 0.90
Mean R^2: 0.88
Train R^2 Score: 0.8985868579151443
Test R^2 Score: 0.8797864074754093
[(0, 'school2'), (0, 'sex2'), (0, 'address2'), (-0.40016172888413376, 'schoolsup2'), (0, 'higher2'), (0, 'internet2'), (0.9385614704549294, 'avg_Grade'), (0, 'Dalc'), (0, 'failures'), (0, 'Medu'), (0, 'studytime')]
[0.8445050520613454, 0.7875576494934906, 0.912704678208583, 0.028291670600175015, 0.7036905045892259, 0.669148905290768, 7.514861031832456e-51, 0.9556040379158879, 0.4691867039354767, 0.4100316438920961, 0.28850470332002875]
[0.19613429748514075, -0.26948348483278056, 0.10962765388534702, -2.193218078782363, 0.3803434252140608, 0.4273163779597795, 14.99845469879079, -0.05567082977866756, -0.7238029729130055, 0.823837945165534, 1.0614076026593409]


After  adding reguralization the overfitting is much less